In [1]:
import pandas as pd

In [2]:
rawTable = pd.read_csv('../data/allHeadlinesNLPRaw.csv')

In [39]:
def makeCount(col):
    s = {}
    for st in rawTable['persons_raw']:
        if not pd.isnull(st):
            for n in st.split(','):
                if n in s.keys():
                    s[n] += 1
                else:
                    s[n] = 1
    return s

def partialNames(nameCount):
    parNames = {}
    for n in nameCount.keys():
        parts = n.split()
        l = len(parts)
        # general partial names
        if l > 1:
            for i in range(l):
                for j in range(i+1,l+1):
                    p = ' '.join(parts[i:j])
                    if p in nameCount.keys() and p != n:
                        if p in parNames.keys():
                            parNames[p].add(n)
                        else:
                            parNames[p] = set([n])
    for p in parNames.keys():
        parNames[p] = sorted(list(parNames[p]), key=lambda x: nameCount[x], reverse=True)
    return parNames

def longNames(nameCount):
    lNames = {}
    for n in nameCount.keys():
        parts = n.split()
        l = len(parts)
        if l > 2:
            for i in range(l):
                for j in range(i,l):
                    if j-i > 1:
                        p = ' '.join(parts[i:j+1])
                        if p in nameCount.keys() and p != n and nameCount[p] > nameCount[n]:
                            if n in lNames.keys():
                                lNames[n].add(p)
                            else:
                                lNames[n] = set([p])
                        p = parts[i] + ' ' + parts[j]
                        print(n)
                        print(p)
                        if p in nameCount.keys() and p != n and nameCount[p] > nameCount[n]:
                            if n in lNames.keys():
                                lNames[n].add(p)
                            else:
                                lNames[n] = set([p])
    for p in lNames.keys():
        lNames[p] = sorted(list(lNames[p]), key=lambda x: nameCount[x], reverse=True)
    return lNames


def makeConvertDic(parNames):
    convert = {}
    for n in parNames.keys():
        if len(n.split()) == 1: 
            mostCom = parNames[n][0].split()
            # ignore first names
            if mostCom[0] == n:
                convert[n] = None
            # convert last names to most common full name
            if  mostCom[len(mostCom)-1] == n:
                convert[n] = ' '.join(mostCom)
        # reduce long names to two words
        if len(n.split()) > 1:
            for long in parNames[n]:
                convert[long] = n
    return convert
        
def reduceCount(nameCount):
    parNames = partialNames(nameCount)
    convert = makeConvertDic(parNames)
    reducedNameCount = {}
    for n in nameCount.keys():
        if n in convert.keys():
            if convert[n] in reducedNameCount.keys():
                reducedNameCount[convert[n]] += nameCount[n]
            else:
                if convert[n] is not None:
                    reducedNameCount[convert[n]] = nameCount[n]
        else:
            reducedNameCount[n] = nameCount[n]
    return(reducedNameCount)
        

In [5]:
nameCount = makeCount(rawTable['persons_raw'])
parNames = partialNames(nameCount)
convert = makeConvertDic(parNames)
    

reducedNameCount = {}
for n in nameCount.keys():
    if n in convert.keys():
        if convert[n] in reducedNameCount.keys():
            reducedNameCount[convert[n]] += nameCount[n]
        else:
            if convert[n] is not None:
                reducedNameCount[convert[n]] = nameCount[n]
    else:
        reducedNameCount[n] = nameCount[n]

In [7]:
redNames = pd.DataFrame.from_dict(reducedNameCount, orient='index').sort_values(0, ascending=False)
redNames = redNames[redNames[0]>5]
reducedNameCount['Theresa May']

256

In [45]:
lNames['Beyoncé Jane Chen']

KeyError: 'Beyoncé Jane Chen'

In [34]:
name = 'Kohl'
rawTable[rawTable.apply(lambda x: not pd.isnull(x['persons_raw']) and name in x['persons_raw'], axis=1)]

,author,date,journal,link,section,summary,title,locations_raw,organizations_raw,persons_raw
24236,THE ASSOCIATED PRESS,2016-05-13,NYT,http://www.nytimes.com/2016/05/13/business/dai...,Business Day,Technology and health care stocks took the lar...,Market Is Mixed After Unemployment Claims Rise,NaN,NaN,Kohl
29160,NaN,2016-06-12,NYT,http://www.nytimes.com/2016/06/12/sports/roger...,Sports,"On grass, the Swiss Federer lost to the Austri...",Roger Federer Falls on His Favorite Surface to...,Germany,NaN,"Federer,Roger Federer,Philipp Kohlschreiber,Do..."
43720,THE ASSOCIATED PRESS,2016-08-11,NYT,http://www.nytimes.com/aponline/2016/08/11/bus...,Business Day,Stocks that moved substantially or traded heav...,"Macy's, Kohl's and Brinker Climb, While Shake ...",NaN,"Macy's,Brinker Climb",Kohl
44013,THE ASSOCIATED PRESS,2016-08-11,NYT,http://www.nytimes.com/aponline/2016/08/11/bus...,Business Day,Kohl's fiscal second-quarter profit beat analy...,Kohl's 2Q Profit Tops Expectations as It Lower...,NaN,NaN,Kohl
44029,REUTERS,2016-08-11,NYT,http://www.nytimes.com/reuters/2016/08/11/busi...,Business Day,All three major U.S. stock indexes closed at r...,Three Major Indexes End at Records for First T...,U.S.,Macy's,Kohl
44030,REUTERS,2016-08-11,NYT,http://www.nytimes.com/reuters/2016/08/11/busi...,Business Day,Kohl's Corp reported a higher-than-expected qu...,Kohl's Quarterly Profit Beats Estimates; Share...,NaN,Corp,Kohl
45198,REUTERS,2016-08-12,NYT,http://www.nytimes.com/2016/08/12/business/dai...,Business Day,"The department store operator Macy’s, which sa...","Stocks End at Record Highs, Bolstered by Retai...",NaN,Macy's,Kohl
45390,REUTERS,2016-08-13,NYT,http://www.nytimes.com/reuters/2016/08/13/spor...,Sports,India crushed West Indies at the third test in...,Kohli Joy as India Rout West Indies to Win Series,"St Lucia,West Indies,India,India Rout West Indies",NaN,"Kohli Joy,Virat Kohli"
63264,THE ASSOCIATED PRESS,2016-09-19,NYT,http://www.nytimes.com/aponline/2016/09/19/bus...,Business Day,"Kohl's plans to hire more than 69,000 addition...","Kohl's to Hire 69,000 Seasonal Workers",NaN,NaN,Kohl
63297,REUTERS,2016-09-19,NYT,http://www.nytimes.com/reuters/2016/09/19/busi...,Business Day,Department store operator Kohl's Corp said on ...,Kohl's to Keep Seasonal Hiring Little Changed ...,United States,"Corp,Target Corp",Kohl


In [8]:
len(pd.unique(rawTable['date']))

366

In [12]:
g = rawTable.groupby(['date', 'journal'])

In [53]:
l=4
for i in range(l):
    for j in range(i+1, l+1):
        print(str(i) + ', ' + str(j))
        print(str(list(range(i,j))))

0, 1
[0]
0, 2
[0, 1]
0, 3
[0, 1, 2]
0, 4
[0, 1, 2, 3]
1, 2
[1]
1, 3
[1, 2]
1, 4
[1, 2, 3]
2, 3
[2]
2, 4
[2, 3]
3, 4
[3]


In [52]:
max([2,3,4,5])

5

In [57]:
s = set([1,2,3])

In [61]:
s()

1

In [62]:
s

{2, 3, 4}